<a href="https://colab.research.google.com/github/sakshamjn/Omdena-OVAC/blob/main/Omdena_text_categorization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
# print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.
import itertools
import os

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn.metrics import confusion_matrix

from tensorflow import keras
layers = keras.layers
models = keras.models


# This code was tested with TensorFlow v1.8
print("You have TensorFlow version", tf.__version__)

You have TensorFlow version 2.3.0


## Get the data
The data has been exported from BigQuery and is stored on GCS, and can be access directly via 

  `!gsutil cp gs://dataset-uploader/bbc/bbc-text.csv .`

It can also be accessed via https://storage.googleapis.com/dataset-uploader/bbc/bbc-text.csv 

In [ ]:
data = pd.read_csv("/content/df2_4_model.csv")

FileNotFoundError: ignored

In [ ]:
data.head()

NameError: ignored

In [ ]:
data['Article'] = data['Article'].astype(str)
data['Class'] = data['Class'].astype(str)

NameError: ignored

In [ ]:
print(data["Article"].dtypes)

NameError: ignored

In [ ]:
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
stop = stopwords.words('english')

# It makes a list of the articels with stopwords removed.
#data['cleaned_articles'] = data['Article'].apply(lambda x: [item for item in str(x).split() if item not in stop])

data['cleaned_articles'] = data["Article"].apply(lambda words: ' '.join(word.lower() for word in words.split() if word not in stop))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


NameError: ignored

In [ ]:
data['Article']

NameError: ignored

In [ ]:
data['cleaned_articles']

0      sending half-naked photo new facebook admirer,...
1      shutting children away behind closed doors, aw...
2      enticed pretty face sent facebook friend reque...
3      a schoolboy came police broken heart. he utter...
4      vancouver - a former canadian schoolteacher sp...
                             ...                        
204    just half women young girls surveyed complaine...
205    thailand seen "severe" restrictions freedom ex...
206    it said one else world could ever love child p...
207    millions thai public school students headed ba...
208    for thais, monday emotionally tumultuous day p...
Name: cleaned_articles, Length: 209, dtype: object

In [ ]:
print(data.dtypes)

NameError: ignored

In [ ]:
data['Class'].value_counts()

NameError: ignored

In [ ]:
train_size = int(len(data) * .8)
print ("Train size: %d" % train_size)
print ("Test size: %d" % (len(data) - train_size))

Train size: 167
Test size: 42


In [ ]:
import pandas as pd 
from sklearn.feature_extraction.text import TfidfTransformer 
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
#instantiate CountVectorizer() 
cv=CountVectorizer() 
 
# this steps generates word counts for the words in your docs 
word_count_vector=cv.fit_transform(data['cleaned_articles'])

NameError: ignored

In [ ]:
word_count_vector.shape

NameError: ignored

In [ ]:
tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True) 
tfidf_transformer.fit(word_count_vector)

TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)

In [ ]:
# print idf values 
df_idf = pd.DataFrame(tfidf_transformer.idf_, index=cv.get_feature_names(),columns=["idf_weights"]) 
 
# sort ascending 
df_idf.sort_values(by=['idf_weights'])

,idf_weights
the,1.043803
said,1.110666
also,1.310155
year,1.384263
people,1.405465
...,...
jaruwat,5.653960
jaruvat,5.653960
jared,5.653960
janet,5.653960


In [ ]:
# count matrix 
count_vector=cv.transform(data['cleaned_articles']) 
 
# tf-idf scores 
tf_idf_vector=tfidf_transformer.transform(count_vector)

NameError: ignored

In [ ]:
feature_names = cv.get_feature_names() 
 
#get tfidf vector for first document 
first_document_vector=tf_idf_vector[0] 
 
#print the scores 
df_tfidf = pd.DataFrame(first_document_vector.T.todense(), index=feature_names, columns=["tfidf"]) 
df_tfidf.sort_values(by=["tfidf"],ascending=False)

NotFittedError: ignored

In [ ]:
first_document_vector.shape

NameError: ignored

In [ ]:
from sklearn.model_selection import train_test_split

train_text, test_text, train_cat, test_cat = train_test_split(data['cleaned_articles'], 
                                                    data['Class'].astype(str), 
                                                    train_size=0.80, 
                                                    random_state=42,
                                                    stratify=data['Class'].astype(str))

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.text import text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences

In [ ]:
MAX_NB_WORDS = 12000
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)

tokenizer.fit_on_texts(data['cleaned_articles'])

In [ ]:
train_text[15]

'chromebooks quickly becoming go-to low-cost laptops preschool primary-level students. but increased access internet comes problems ranging security threats cyberbullying. israeli software company check point developed chrome extension called sandblast mitigate threats facing school-aged youngsters. the extension beefs cybersecurity chromebooks, also tackles bullying problem blocking keywords issuing alerts students post offensive content. "sandblast education monitors controls browsing activity using advanced, dynamic highly accurate url filtering capabilities granular whitelisting blacklisting capabilities," said bruce chai, head threat prevention check point asia. "it blocks explicit search results enforcing safe-search capabilities google, yahoo, bing youtube, block access search engines." a study us found nearly 90% students aged 4 12 witnessed peers get cyberbullied, 90% children aged 8 16 exposed internet pornography. the sandblast technology differs leading competitors providin

In [ ]:
tokenizer.texts_to_sequences([train_text[15]])

[[3994,
  2832,
  1324,
  340,
  138,
  1359,
  2263,
  3334,
  6889,
  1826,
  504,
  32,
  37,
  852,
  85,
  13,
  368,
  336,
  2880,
  142,
  366,
  31,
  5128,
  1274,
  342,
  1019,
  490,
  1332,
  6890,
  5129,
  110,
  3995,
  3823,
  366,
  931,
  20,
  167,
  1151,
  2,
  5129,
  6891,
  3192,
  3994,
  6,
  5130,
  27,
  106,
  6892,
  6893,
  5131,
  3228,
  32,
  162,
  1570,
  76,
  3995,
  93,
  5132,
  2534,
  5133,
  651,
  290,
  2426,
  6894,
  1941,
  6895,
  6896,
  6897,
  2264,
  6898,
  6899,
  6900,
  2264,
  1,
  6901,
  3996,
  338,
  944,
  817,
  1019,
  490,
  271,
  22,
  5134,
  424,
  255,
  729,
  3997,
  116,
  255,
  2264,
  483,
  6902,
  6903,
  2881,
  1189,
  85,
  255,
  3998,
  43,
  184,
  33,
  52,
  577,
  2495,
  32,
  167,
  544,
  225,
  3335,
  1469,
  92,
  2826,
  2495,
  3,
  167,
  667,
  345,
  898,
  13,
  67,
  2,
  3995,
  283,
  5135,
  722,
  5136,
  746,
  178,
  366,
  6904,
  929,
  988,
  32,
  41,
  111,
  6905,
  698,
 

In [ ]:
train_sequences = tokenizer.texts_to_sequences(train_text)
test_sequences = tokenizer.texts_to_sequences(test_text)

In [ ]:
len(train_sequences)

167

In [ ]:
len(train_sequences[1])

447

In [ ]:
MAX_LENGTH = 10000
padded_train_sequences = pad_sequences(train_sequences, maxlen=MAX_LENGTH)
padded_test_sequences = pad_sequences(test_sequences, maxlen=MAX_LENGTH)

In [ ]:
padded_train_sequences

array([[   0,    0,    0, ..., 6363,  626, 6312],
       [   0,    0,    0, ..., 1992,  180,  162],
       [   0,    0,    0, ..., 5520, 5537, 1364],
       ...,
       [   0,    0,    0, ..., 2892,  395,   77],
       [   0,    0,    0, ..., 1433,  271, 2240],
       [   0,    0,    0, ..., 1203,  239, 3431]], dtype=int32)

In [ ]:
padded_test_sequences

array([[    0,     0,     0, ...,    85,   488,  1086],
       [    0,     0,     0, ...,  1397,   369,  1496],
       [    0,     0,     0, ...,  1615,  2003,    21],
       ...,
       [    0,     0,     0, ...,   451,  1084,  2385],
       [    0,     0,     0, ...,   150,   329,   491],
       [    0,     0,     0, ...,  1665, 10943,  3424]], dtype=int32)

In [ ]:
## def train_test_split(data, train_size):
#    train = data[:train_size]
#   test = data[train_size:]
#    return train, test

## Data preparation
There's some work to be done in order for our data to be ready for training.
1. First we'll split the data into training and test sets.
1. Then we'll tokenize the words (text), and then convert them to a numbered index. 
1. Next we'll do the same for the labels (categories), by using the `LabelEncoder` utility.
1. Finally, we'll convert the labels to a one-hot representation.

In [ ]:
## train_cat, test_cat = train_test_split(data['Class'], train_size)
#train_text, test_text = train_test_split(data['Article'], train_size)

In [ ]:
from sklearn.model_selection import train_test_split

train_text, test_text, train_cat, test_cat = train_test_split(data['cleaned_articles'], 
                                                    data['Class'].astype(str), 
                                                    train_size=0.70, 
                                                    random_state=42,
                                                    stratify=data['Class'].astype(str))

NameError: ignored

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer_word = TfidfVectorizer(max_features=40000,
                             min_df=5, 
                             max_df=0.5, 
                             analyzer='word', 
                             stop_words='english', 
                             ngram_range=(1, 2))

vectorizer_word.fit(train_text)

tfidf_matrix_word_train = vectorizer_word.transform(train_text)
tfidf_matrix_word_test = vectorizer_word.transform(test_text)

NameError: ignored

In [ ]:
tfidf_matrix_word_train

NameError: ignored

In [ ]:
tfidf_matrix_word_test

NameError: ignored

In [ ]:
max_words = 20000
tokenize = keras.preprocessing.text.Tokenizer(num_words=max_words, 
                                              char_level=False)

NameError: ignored

In [ ]:
tokenize.fit_on_texts(train_text) # fit tokenizer to our training text data
x_train = tokenize.texts_to_matrix(tfidf_matrix_word_train)
x_test = tokenize.texts_to_matrix(test_text)

NameError: ignored

In [ ]:
# Use sklearn utility to convert label strings to numbered index
encoder = LabelEncoder()
encoder.fit(train_cat)
y_train = encoder.transform(train_cat)
y_test = encoder.transform(test_cat)

NameError: ignored

In [ ]:
# Converts the labels to a one-hot representation
num_classes = np.max(y_train) + 1
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

NameError: ignored

In [ ]:
# Inspect the dimenstions of our training and test data (this is helpful to debug)
print('x_train shape:', padded_train_sequences.shape)
print('x_test shape:', padded_test_sequences.shape)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

x_train shape: (146, 10000)
x_test shape: (63, 10000)
y_train shape: (146, 3)
y_test shape: (63, 3)


In [ ]:
  # Average number of words per review 
tr_l = [len(x.split()) for x in padded_train_sequences]
te_l = [len(x.split()) for x in padded_test_sequences]
print("Train Sequence length distribution:\n")
print(pd.Series(tr_l).describe())
print("\n\nTest Sequence length distribution:\n")
print(pd.Series(te_l).describe())

NameError: ignored

## Train the model

Build the model using Keras layers and hyperparameters of your choosing. Then call `model.fit()`

In [ ]:
# This model trains very quickly and 2 epochs are already more than enough
# Training for more epochs will likely lead to overfitting on this dataset
# You can try tweaking these hyperparamaters when using this model with your own data
batch_size = 4
epochs = 2
drop_ratio = 0.1

In [ ]:
from sklearn.metrics import accuracy_score

max_words = 10000
# Build the model
model = models.Sequential()
model.add(layers.Dense(1024, input_shape=(max_words,)))
model.add(layers.Activation('relu'))
model.add(layers.Dropout(drop_ratio))
model.add(layers.Dense(512, input_shape=(max_words,)))
model.add(layers.Activation('relu'))
model.add(layers.Dropout(drop_ratio))
model.add(layers.Dense(256, input_shape=(max_words,)))
model.add(layers.Activation('relu'))
model.add(layers.Dropout(drop_ratio))
model.add(layers.Dense(128, input_shape=(max_words,)))
model.add(layers.Activation('relu'))
model.add(layers.Dropout(drop_ratio))
model.add(layers.Dense(num_classes))
model.add(layers.Activation('softmax'))

print("Hello")

from keras import metrics
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=[metrics.Precision()])
              #metrics=[metrics.categorical_accuracy])
              #metrics=[keras.metrics.Precision()])

NameError: ignored

LSTM

In [ ]:

from keras.layers import Dense, Dropout, Embedding, LSTM, GlobalMaxPooling1D, SpatialDropout1D
max_words = 10000
max_phrase_len = 10000

model_lstm = models.Sequential()

model_lstm.add(Embedding(input_dim = max_words, output_dim = 256, input_length = max_phrase_len))
model_lstm.add(SpatialDropout1D(0.3))
model_lstm.add(LSTM(256, dropout = 0.3, recurrent_dropout = 0.3))
model_lstm.add(Dense(256, activation = 'relu'))
model_lstm.add(Dropout(0.3))
model_lstm.add(Dense(num_classes, activation = 'softmax'))

model_lstm.compile(
    loss='categorical_crossentropy',
    optimizer='Adam',
    metrics=['accuracy']
)

NameError: ignored

In [ ]:
history = model_lstm.fit(
    padded_train_sequences,
    y_train,
    validation_split = 0.1,
    epochs = 100,
    batch_size = 512,
    callbacks = None,
    shuffle = True,
    steps_per_epoch = None,
    workers = 1,
    class_weight = None)


NameError: ignored

In [ ]:
padded_train_sequences

NameError: ignored

In [ ]:
y_train

NameError: ignored

In [ ]:
# model.fit trains the model
# The validation_split param tells Keras what % of our training data should be used in the validation set
# You can see the validation loss decreasing slowly when you run this
# Because val_loss is no longer decreasing we stop training to prevent overfitting
history = model.fit(padded_train_sequences, y_train,
                    batch_size=batch_size,
                    epochs=100,
                    verbose=1,
                    validation_split=0.10)

NameError: ignored

## Evaluate the model
Evaluation is easy. Just call `model.evaluate()`.

In [ ]:
# Evaluate the accuracy of our trained model
score = model.evaluate(padded_test_sequences, y_test,
                       batch_size=batch_size, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

NameError: ignored

## Hyperparameter tuning

[link text](https:// [link text](https://))This is a good time to go back and tweak some parameters such as `epoch`, `batch size`, `dropout ratio`, network structure, activation function, and others, to see if you can improve the accuracy.

In this particular case, to make it more challenging, I recommend reducing the max words of the call to `keras.preprocessing.text.Tokenizer`. This will reduce the number of words for each input sample, thus making it more challenging to accurately predict the category. (Notice that not all hyperparameters are necessarily inside the model. This is one such example.)

The default was up to 1000 words per article. See what happens when you reduce that number to 200 words, or 50 words, or even fewer. As the evaluation accuracy drops, the effects of your hyperparameter tuning will be more pronounced, with successful adjustments making meaningful improvements to the model performance.

To make this process easier to manage, I've encapulated the model definition and training and evaluation calls into one function call. You can add additional parameters as needed.

In [ ]:
def run_experiment(batch_size, epochs, drop_ratio):
  print('batch size: {}, epochs: {}, drop_ratio: {}'.format(
      batch_size, epochs, drop_ratio))
  model = models.Sequential()
  model.add(layers.Dense(512, input_shape=(max_words,)))
  model.add(layers.Activation('relu'))
  model.add(layers.Dropout(drop_ratio))
  model.add(layers.Dense(num_classes))
  model.add(layers.Activation('softmax'))

  model.compile(loss='categorical_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])
  history = model.fit(padded_test_sequences, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=0,
                    validation_split=0.1)
  score = model.evaluate(padded_test_sequences, y_test,
                       batch_size=batch_size, verbose=0)
  print('\tTest loss:', score[0])
  print('\tTest accuracy:', score[1])

In [ ]:
batch_size = 16
epochs = 40
drop_ratio = 0.4
run_experiment(batch_size, epochs, drop_ratio)

batch size: 16, epochs: 40, drop_ratio: 0.4


NameError: ignored

###Hyperparameter Search

You can also automate this process using for-loops and more sophiscated methods of deciding which combinations of hyperparameter values to try out.

Exhaustive search is generally not the most elegant way, this is mostly just for illustrative purposes.

In [ ]:
# Note: The data processed for this output had the max text length set to 400.
# for batch_size in range(10,31,10):
#   for epochs in range(3,15,5):
#     for drop_ratio in np.linspace(0.1, 0.5, 3):
#       run_experiment(batch_size, epochs, drop_ratio)

## Make some predictions
Take some samples from the test dataset and inspect some individual predictions, to ensure that things are sensible.


In [ ]:
# Here's how to generate a prediction on individual examples
text_labels = encoder.classes_ 

for i in range(10):
    prediction = model.predict(np.array([padded_test_sequences[i]]))
    predicted_label = text_labels[np.argmax(prediction)]
    print(test_text.iloc[i][:50], "...")
    print('Actual label:' + test_cat.iloc[i])
    print("Predicted label: " + predicted_label + "\n")  

NameError: ignored

## (optional) Extra extra! Visualize the confusion matrix
This can help identify which areas were a challenge to get right, if the model is performing poorly.

In [ ]:
y_softmax = model.predict(padded_test_sequences)

y_test_1d = []
y_pred_1d = []

for i in range(len(y_test)):
    probs = y_test[i]
    index_arr = np.nonzero(probs)
    one_hot_index = index_arr[0].item(0)
    y_test_1d.append(one_hot_index)

for i in range(0, len(y_softmax)):
    probs = y_softmax[i]
    predicted_index = np.argmax(probs)
    y_pred_1d.append(predicted_index)

NameError: ignored

In [ ]:
# This utility function is from the sklearn docs: http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html
def plot_confusion_matrix(cm, classes,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """

    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title, fontsize=30)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45, fontsize=22)
    plt.yticks(tick_marks, classes, fontsize=22)

    fmt = '.2f'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label', fontsize=25)
    plt.xlabel('Predicted label', fontsize=25)

NameError: ignored

In [ ]:
cnf_matrix = confusion_matrix(y_test_1d, y_pred_1d)
plt.figure(figsize=(24,20))
plot_confusion_matrix(cnf_matrix, classes=text_labels, title="Confusion matrix")
plt.show()

NameError: ignored

In [ ]:
#from sklearn.metrics import average_precision_score
#average_precision = average_precision_score(y_test_1d, y_pred_1d)

#print('Average precision-recall score: {0:0.2f}'.format(average_precision))

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

In [ ]:
print(confusion_matrix(y_test_1d, y_pred_1d))
print(classification_report(y_test_1d, y_pred_1d)) 

[[ 5  5  1]
 [10 27  2]
 [ 4  8  1]]
              precision    recall  f1-score   support

           0       0.26      0.45      0.33        11
           1       0.68      0.69      0.68        39
           2       0.25      0.08      0.12        13

    accuracy                           0.52        63
   macro avg       0.40      0.41      0.38        63
weighted avg       0.52      0.52      0.51        63

